# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
from azureml.core import Experiment, Model, Webservice
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
import pandas as pd

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

ws = Workspace.from_config()
experiment_name = 'demo-hyperdrive-experiment'

experiment=Experiment(ws, experiment_name)

# Prepare the datastore to upload data
datastore = ws.get_default_datastore()

# Set the upload location to target_path in datastore
datastore.upload(src_dir='./', target_path = 'data')

Uploading an estimated of 9 files
Target already exists. Skipping upload for data/automl.ipynb
Target already exists. Skipping upload for data/automl.ipynb.amltmp
Target already exists. Skipping upload for data/hyperparameter_tuning.ipynb
Target already exists. Skipping upload for data/score.py
Target already exists. Skipping upload for data/train.py
Target already exists. Skipping upload for data/.ipynb_aml_checkpoints/automl-checkpoint2021-0-29-11-2-34.ipynb
Target already exists. Skipping upload for data/.ipynb_aml_checkpoints/hyperparameter_tuning-checkpoint2021-0-29-11-3-8.ipynb
Uploading ./automl.log
Uploaded ./automl.log, 1 files out of an estimated total of 7
Uploading ./azureml_automl.log
Uploaded ./azureml_automl.log, 2 files out of an estimated total of 7
Uploaded 2 files


$AZUREML_DATAREFERENCE_10c46dcf4e3047c9a1ef698d73bf0baa

In [3]:
dataset = Dataset.get_by_name(ws, name = "bankchurners")
dataset

{
  "source": [
    "('workspaceblobstore', 'UI/01-29-2021_110643_UTC/BankChurners.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "fd67c9bf-6db3-4b3d-8f2a-63297843ca70",
    "name": "bankchurners",
    "version": 1,
    "workspace": "Workspace.create(name='quick-starts-ws-136499', subscription_id='6b4af8be-9931-443e-90f6-c4c34a1f9737', resource_group='aml-quickstarts-136499')"
  }
}

In [4]:
compute_name = "demo-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    # Create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# Use get_status() to get a detailed status for the current cluster
print(compute_target.get_status().serialize())

Found existing cluster, use it.

Running
{'errors': [], 'creationTime': '2021-01-29T11:02:47.998719+00:00', 'createdBy': {'userObjectId': '0a853285-832a-44a3-84c4-95d5b9856dd9', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': 'ODL_User 136499'}, 'modifiedTime': '2021-01-29T11:06:21.408812+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS3_V2'}


## Project Directory

Now that we have your data and training script prepared, we are ready to train on your remote compute. We will take advantage of Azure compute to leverage a CPU cluster.

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

In [5]:
import os
import shutil

project_folder = './sklearn-bankchurners'
os.makedirs(project_folder, exist_ok=True)

# Copy prepared training script into project directory
shutil.copy('train.py', project_folder)

'./sklearn-bankchurners/train.py'

## Create Environment

Define a conda environment YAML file with our training script dependencies and create an Azure ML environment.

In [6]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults

Writing conda_dependencies.yml


In [7]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

## Hyperdrive Configuration

TODO: Explain the model you are using.

In [8]:
from azureml.core import ScriptRunConfig

#Create the estimator and hyperdrive config
src = ScriptRunConfig(source_directory= project_folder, 
                script="train.py",
                arguments=['--kernel', 'linear', '--penalty', 1.0],
                compute_target= compute_target,
                environment = sklearn_env)

## Submit job

Run your experiment by submitting your ScriptRunConfig object. Note that this call is asynchronous.

In [9]:
from azureml.widgets import RunDetails

run = experiment.submit(src)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [10]:
run.wait_for_completion(show_output=True)

RunId: demo-hyperdrive-experiment_1611918846_1af4ef1b
Web View: https://ml.azure.com/experiments/demo-hyperdrive-experiment/runs/demo-hyperdrive-experiment_1611918846_1af4ef1b?wsid=/subscriptions/6b4af8be-9931-443e-90f6-c4c34a1f9737/resourcegroups/aml-quickstarts-136499/workspaces/quick-starts-ws-136499

Streaming azureml-logs/20_image_build_log.txt

2021/01/29 11:14:19 Downloading source code...
2021/01/29 11:14:20 Finished downloading source code
2021/01/29 11:14:21 Creating Docker network: acb_default_network, driver: 'bridge'
2021/01/29 11:14:21 Successfully set up Docker network: acb_default_network
2021/01/29 11:14:21 Setting up Docker configuration...
2021/01/29 11:14:22 Successfully set up Docker configuration
2021/01/29 11:14:22 Logging in to registry: 16848a390a2a4230a4a0aa21a12e2333.azurecr.io
2021/01/29 11:14:24 Successfully logged into 16848a390a2a4230a4a0aa21a12e2333.azurecr.io
2021/01/29 11:14:24 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', N

{'runId': 'demo-hyperdrive-experiment_1611918846_1af4ef1b',
 'target': 'demo-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-29T11:19:15.002475Z',
 'endTimeUtc': '2021-01-29T11:28:09.470148Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '643c524f-70b2-476c-88b6-12fa3fd7673b',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--kernel', 'linear', '--penalty', '1'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'demo-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'environment': {'name': 'sklearn-env',
   'version': 'Autosave_2021

## Tune model hyperparameters

We will optimize our model's hyperparameters using Azure Machine Learning's hyperparameter tuning capabilities.

TODO: Explain the reason for chosing the different hyperparameters, termination policy and config settings.

In [11]:
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice

# Create an early termination policy. This is not required if using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)
    
# Create the different params that will be used during training
param_sampling = RandomParameterSampling( {
    "--kernel": choice('linear', 'rbf', 'poly', 'sigmoid'),
    "--penalty": choice(0.5, 1, 1.5)
    }
)

hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4,
                                     policy=early_termination_policy)

In [12]:
# Submit the experiment
hyperdrive_run = experiment.submit(hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_2dccf12d-bac5-42a1-aedd-b1135a172cd0
Web View: https://ml.azure.com/experiments/demo-hyperdrive-experiment/runs/HD_2dccf12d-bac5-42a1-aedd-b1135a172cd0?wsid=/subscriptions/6b4af8be-9931-443e-90f6-c4c34a1f9737/resourcegroups/aml-quickstarts-136499/workspaces/quick-starts-ws-136499

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-29T11:29:05.376079][API][INFO]Experiment created<END>\n""<START>[2021-01-29T11:29:05.928117][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-29T11:29:06.391828][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-29T11:29:06.8812983Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

best_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_metrics['Accuracy'])

In [ ]:
# List the model files uploaded during the run

print(best_run.get_file_names())

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name='sklearn-bankchurners', model_path='outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service